In [1]:
# Importing required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from featureExtraction import getDomain, getURL_Length, getDomain_Length, getURL_Depth, getURL_IP, getURL_AtSymbol, getURLDomain_HTTP
from featureExtraction import getURL_RedirectPosition, getURL_HyphenInDomain, getURL_Dot, getURL_HTTPs, getURL_ASCII, getURL_Digits

In [2]:
# Loading the data
urlData = pd.read_csv("data/data_bal20000.csv")
urlData.head()

,Labels,URLs
0,1,http://dbs.vote-friend.com/sg?ref=anything
1,0,https://www.reynoldstransfer.com/versa-lift-fo...
2,1,https://www.halisupportservice.com/Login.php
3,0,https://www.signets.com.br/wp-includes/wlwmani...
4,1,https://docs.google.com/document/d/e/2PACX-1vT...


In [3]:
'''
urlData['Length'] = urlData['URLs'].tail(150).str.len()

sns.set_style("whitegrid")
g = sns.catplot(data=urlData.tail(150), x="URLs", y="Length", hue="Labels", kind="bar")
g.set_axis_labels("URLs", "Length")

g.fig.set_figheight(6)
g.fig.set_figwidth(12)

g.set(xticks=[])
plt.ylim(40, 60)

del urlData["Length"]
'''

'\nurlData[\'Length\'] = urlData[\'URLs\'].tail(150).str.len()\n\nsns.set_style("whitegrid")\ng = sns.catplot(data=urlData.tail(150), x="URLs", y="Length", hue="Labels", kind="bar")\ng.set_axis_labels("URLs", "Length")\n\ng.fig.set_figheight(6)\ng.fig.set_figwidth(12)\n\ng.set(xticks=[])\nplt.ylim(40, 60)\n\ndel urlData["Length"]\n'

In [4]:
'''
urlData['Depth'] = urlData['URLs'].tail(150).str.count('/')

sns.set_style("whitegrid")
g = sns.catplot(data=urlData.tail(150), x="URLs", y="Depth", hue="Labels", kind="bar")
g.set_axis_labels("URLs", "Depth")

g.fig.set_figheight(5)
g.fig.set_figwidth(12)

g.set(xticks=[])
plt.ylim(0, 10)

del urlData["Depth"]
'''

'\nurlData[\'Depth\'] = urlData[\'URLs\'].tail(150).str.count(\'/\')\n\nsns.set_style("whitegrid")\ng = sns.catplot(data=urlData.tail(150), x="URLs", y="Depth", hue="Labels", kind="bar")\ng.set_axis_labels("URLs", "Depth")\n\ng.fig.set_figheight(5)\ng.fig.set_figwidth(12)\n\ng.set(xticks=[])\nplt.ylim(0, 10)\n\ndel urlData["Depth"]\n'

In [5]:
'''
urlData['DigitsNo'] = urlData['URLs'].tail(150).str.count('\d')

sns.set_style("whitegrid")
g = sns.catplot(data=urlData.tail(150), x="URLs", y="DigitsNo", hue="Labels", kind="bar")
g.set_axis_labels("URLs", "DigitsNo")

g.fig.set_figheight(5)
g.fig.set_figwidth(12)

g.set(xticks=[])
plt.ylim(0, 20)

del urlData["DigitsNo"]
'''

'\nurlData[\'DigitsNo\'] = urlData[\'URLs\'].tail(150).str.count(\'\\d\')\n\nsns.set_style("whitegrid")\ng = sns.catplot(data=urlData.tail(150), x="URLs", y="DigitsNo", hue="Labels", kind="bar")\ng.set_axis_labels("URLs", "DigitsNo")\n\ng.fig.set_figheight(5)\ng.fig.set_figwidth(12)\n\ng.set(xticks=[])\nplt.ylim(0, 20)\n\ndel urlData["DigitsNo"]\n'

In [6]:
'''
getDomain(urlData)

urlData['DomainLength'] = urlData['Domain'].tail(150).str.len()

sns.set_style("whitegrid")
g = sns.catplot(data=urlData.tail(150), x="Domain", y="DomainLength", hue="Labels", kind="bar")
g.set_axis_labels("Domain", "DomainLength")

g.fig.set_figheight(6)
g.fig.set_figwidth(12)

g.set(xticks=[])
plt.ylim(0, 40)

del urlData["DomainLength"]

'''

'\ngetDomain(urlData)\n\nurlData[\'DomainLength\'] = urlData[\'Domain\'].tail(150).str.len()\n\nsns.set_style("whitegrid")\ng = sns.catplot(data=urlData.tail(150), x="Domain", y="DomainLength", hue="Labels", kind="bar")\ng.set_axis_labels("Domain", "DomainLength")\n\ng.fig.set_figheight(6)\ng.fig.set_figwidth(12)\n\ng.set(xticks=[])\nplt.ylim(0, 40)\n\ndel urlData["DomainLength"]\n\n'

In [7]:
# Feature extraction

# 1. Domain name
getDomain(urlData)

# 2. URL length
getURL_Length(urlData)

# 3. Domain length
getDomain_Length(urlData)

# 3. URL depth (number of slashes)
getURL_Depth(urlData)

# 4. URL @ symbol presence
getURL_AtSymbol(urlData)

# 5. IP address presence in URL
getURL_IP(urlData)

# 6. Transport layer security (TLS) presence in URL
getURL_HTTPs(urlData)

# 7. HTTP/HTTPS presence in URL Domain
getURLDomain_HTTP(urlData)

# 8. // Redirect postion in URL
getURL_RedirectPosition(urlData)

# 9. Prefix and suffix '-'
getURL_HyphenInDomain(urlData)

# 10. Number of '.' in URL Domain
getURL_Dot(urlData)

# 11. Unicode characters in URL
getURL_ASCII(urlData)

# 12. Number of digits in URL
getURL_Digits(urlData)

urlData.head(10)


c:\Users\Or3a\Desktop\UNIBZ\Sem 2\Machine Learning\ML Project\featureExtraction.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  Domain = Domain.str.replace('www.', '')


,Labels,URLs,Domain,URL_Length,Domain_Length,URL_depth,URL_atSymbol,URL_IP,URL_HTTPS,URL_DomainHTTP,URL_RedirectPosition,URL_HyphenInDomain,URL_DotsNo,URL_NonASCHII,URL_Digits
19990,0,https://www.eclipse-print.com/sticker,eclipse-print.com,Legitimate,Suspicious,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Fishing,Legitimate,Legitimate,Legitimate
19991,0,https://www.hase-kramer.at/wp-includes/wlwmani...,hase-kramer.at,Suspicious,Suspicious,Suspicious,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Fishing,Legitimate,Legitimate,Legitimate
19992,1,http://modernism.gistemp.com/mail/en.php,modernism.gistemp.com,Legitimate,Fishing,Suspicious,Legitimate,Legitimate,Fishing,Legitimate,Fishing,Legitimate,Suspicious,Legitimate,Legitimate
19993,0,https://www.cartrack.ng/,cartrack.ng,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate
19994,1,https://www.authorise-mydevice.org/Login.php,authorise-mydevice.org,Legitimate,Fishing,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Fishing,Legitimate,Legitimate,Legitimate
19995,1,https://messagerievocale56.wixsite.com/orange,messagerievocale56.wixsite.com,Legitimate,Fishing,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Suspicious,Legitimate,Legitimate
19996,1,https://06e6f4d3bb4140516.temporary.link/pt/DS...,06e6f4d3bb4140516.temporary.link,Fishing,Fishing,Fishing,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Suspicious,Legitimate,Fishing
19997,1,http://lbcpszonaseguravialbcp.cub-medisc.com/1...,lbcpszonaseguravialbcp.cub-medisc.com,Fishing,Fishing,Suspicious,Legitimate,Legitimate,Fishing,Legitimate,Fishing,Fishing,Suspicious,Legitimate,Fishing
19998,0,https://dljproduce.com/produce-commodity-calen...,dljproduce.com,Suspicious,Suspicious,Suspicious,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate
19999,0,https://www.vineinternational.org/team,vineinternational.org,Legitimate,Fishing,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate,Legitimate
